In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import numpy as np
from matplotlib import pyplot as plt

# Introduction xarrays

This is just to get a quick introductino on the structure of the xarray data type.

In the cell below we generate a xarray with dimensions $(3,3)$ for variable $x$ with coordinates $(10,20)$ and $y$ with coordinates $(1,2,3)$.

### data.values

Returns the np.random.randn(2, 3) values you defined when generating the xarray.

### data.dims

Returns the name of the dimensions.

### data.coords
Returns the coordinates for all axis directions with coordinate names and datatype of the coordinates.

### data.attrs

Returns other attributes in form of a dictionary with you can easily add by generating a new value associated with a new key.

In [ ]:
data = xr.DataArray(np.random.randn(2, 3), dims=("x", "y"), coords={"x": [10, 20], "y": [1, 2,3]})
data
# data.values
# data.dims
# data.coords
# data.attrs["key"] = "value"
# data.attrs

### Additional example

The below cell is an additional example just to show that all datatypes can be used when generating/creating the xarrays.

### For more: documentation html:
http://xarray.pydata.org/en/stable/index.html

In [ ]:
xr.DataArray(pd.Series(range(3), index=list("abc"), name="foo"))

# EMC3 data

The prerequisite for this example to work is to have downloaded the file emc3_example.nc and have the libraries specified in this script installed in your enviroment. The emc3_example.nc can be found and downloaded here: https://gitlab.mpcdf.mpg.de/dave/xemc3-data given that you have acces.

The path specified in the string in the cell below is where you have stored the emc3_example.nc locally on your computer.

In [ ]:
import xemc3
import xarray as xr
path = r"C:\Users\joag\Documents\Notebooks\emc3_example.nc"
ds = xr.open_dataset(path)
ds

## dataset (ds) explanation

When running the codeline ds you get an overview of what the xarray object consist of.

### ds.coords['R_bounds']

R_bounds represents the coordinates of the vertices at the gridcells in the radial direction in the $xy$-plane. Here $r = \sqrt{x^2 + y^2}$.

### ds.coords['z_bounds']
z_bounds represents the coordinates of the vertices of the gridcells in the $z$-direction.

In [ ]:
ds.coords['R_bounds']

In [ ]:
ds.coords['z_bounds']

### Toroidal slice
A toroidal slice is defined as the grid of $(r,z)$-values at a fixed angle $\phi$. The values of the $\phi$-angles used in the W7X grid can be found in the paragraph below and demonstrated in the next cell.

### ds.coords['phi_bounds']
Running the cell below gives you an array of the $\phi$ angles.

In [ ]:
ds.coords['phi_bounds']

## ds.emc3.plot_Rz(parameter, phi = $\phi$)

The parameter is given as a string, and the valid parameters is the same as the measurements that is possible to do in W7X. The angle phi $= \phi$ is the angle given in radians as floats. 

The floats of the angle $\phi$ can be found in the dictionary defined by ds.coords['phi_bounds'] which has 2 dimensions; one for the actual angles $\phi$, and another for the change in angle $d\phi$. There are 36 different values for $\phi$ since the reactor has a five-fold symmetry: $5\cdot 36 = 180^{\circ}$.

In the cells below are some examples of the parameter electron temperature $T_e$ plotted in toroidal slices for phi index $n_{\phi} = [0,18,35]$.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
def plot_Te(iy):
    plt.figure(figsize=(20,10))
    ds.emc3.plot_Rz("Te", phi=ds.coords['phi_bounds'][iy][0])
iy = widgets.IntSlider(min = 0, max = 35, value = 0)
interact(plot_Te, iy = iy)

## Preprocessing of gridstructure

The parametervalues are not well defined in each gridcell, but rather at the center or the mean of the vertices of the gridcell: $\mathbf{r}_{param} = \langle \mathbf{r}_{vertex} \rangle$. A simplified analogy is the centerpoint of a 3D cube.

Specifying the dimensions of the mean coordinates by giving ds.direction_bounds.mean(arg) the argument dim=("delta_r", "delta_theta", "delta_phi") you give the mean secify that the mean coordinates the same number of dimensions per axis as the number of cells in each axial direction. Below is an illustration of the situation:

<img src="Cell_visualization.png">

After defining the vertices of the grid at the point where the parameters are actually defined it is possible to do an interpolation such that we get interpolated parameter values at cartesian grid points. The motivation for doing such an interpolation is the simplicity of calculating gradients. The result of the interpolation is equidistant gridpoints in a 3D cartesian grid. This makes it very simple to calculate gradients etc. This is highly non-trivial in the $(r, \theta, \phi)$-mesh.

In [ ]:
R = ds.R_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi"))
z = ds.z_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi"))
phi = ds.phi_bounds.mean(dim="delta_phi")
x = R * np.cos(phi)
y = R * np.sin(phi)
x, ds.Te

In [ ]:
x.shape
tot_grid_points = x.shape[0]*x.shape[1]*x.shape[2]
print("Total number of gridpoints in the W7X gridstructure is ", tot_grid_points)

## Use of NaN values in the parameter mesh

Not all gridcells have a defined parameter value attached to it. This is mostly the outer and inner region of the machine where the values of many parameter has been left out because this is not the regions where the interesting physics happen.
This is illustrated in the above plot example of the electron temperature $T_e$. In the cell below you can se how large a fraction of the total number of gridpoints the parameter mesh for the electron temperature that has NaN as a value.

In [ ]:
n_nans_Te = int(list(np.where(np.isnan(np.asarray(ds.Te)) == True)[0].shape)[0])
print("How many nans in the Te mesh? ", np.where(np.isnan(np.asarray(ds.Te)) == True)[0].shape)
print("Fraction of nans with respect to gridpoints ", n_nans_Te/tot_grid_points)

In [ ]:
np.asarray(ds.Te).shape

## Grid

In the cell below there is an interactive plot of the grid. You can use the slider to iterate through all toroidal slices(all $\phi$ angles).

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
def plot_emc3grid(ip):
    plt.figure(figsize=(15,15))
    ds.emc3.plot_Rz(None, phi=ds.coords['phi_bounds'][ip][0])
ip = widgets.IntSlider(min = 0, max = 35, value = 0)
interact(plot_emc3grid, ip = ip)

## Grid with boundaries

Interactive plot of the grid, here you can use the ip slider to iterate through all toroidal slices(all phi angles),
the rmin and rmax to set the boundaries in r direction, and the zmin and zmax to set the boundaries in z direction.

In [ ]:
# find boundaries of the grid
rmin = np.min(np.asarray(R))
rmax = np.max(np.asarray(R))
zmin = np.min(np.asarray(z))
zmax = np.max(np.asarray(z))

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
def plot_emc3grid(ip, rmin, rmax, zmin, zmax):
    plt.figure(figsize=(100,100))
    ds.emc3.plot_Rz('Te', phi=ds.coords['phi_bounds'][ip][0], Rmin = rmin, Rmax = rmax, zmin = zmin, zmax = zmax)
ip = widgets.IntSlider(min = 0, max = 35, value = 0)
rmin_slider = widgets.FloatSlider(min = rmin, max = rmax, value = rmin, readout_format='.9f')
rmax_slider = widgets.FloatSlider(min = rmin, max = rmax, value =rmax, readout_format='.9f')
zmin_slider = widgets.FloatSlider(min = zmin, max = zmax, value = zmin, readout_format='.9f')
zmax_slider = widgets.FloatSlider(min = zmin, max = zmax, value = zmax, readout_format='.9f')
interact(plot_emc3grid, ip = ip, rmin = rmin_slider, rmax = rmax_slider, zmin = zmin_slider, zmax = zmax_slider)

In [ ]:
idx_zmin = np.where(np.isclose(-0.77468, np.asarray(z), rtol = 1e-6) == True)[0]
idx_zmax = np.where(np.isclose(0.82532, np.asarray(z), rtol = 5e-7) == True)[0]
idx_rmin = np.where(np.isclose(5.65635, np.asarray(R), rtol = 1e-9) == True)[0]
idx_rmax = np.where(np.isclose(5.85635, np.asarray(R), rtol = 1e-7) == True)[0]

print(idx_zmin, idx_zmax, idx_rmin, idx_rmax)

In [ ]:
def plot_Te_zoomed(ip):
    fig = plt.figure()
    ax = fig.add_subplot()
    c = plt.contourf(R[50:60, 450:460,ip],
                     z[50:60, 450:460,ip], 
                     np.array(ds.Te[50:60, 450:460,ip]))
    plt.colorbar(c)
phislider = widgets.IntSlider(min = 0, max = 35)
interact(plot_Te_zoomed, ip = phislider)